In [2]:
import sys
import os
import math
import random
import array
from datetime import datetime

import numpy as np
import pandas as pd 

from scipy.stats import lognorm
from scipy.stats import beta
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import expon
from scipy.stats import pareto
import scipy.sparse as sp
import scipy.linalg as la
from scipy.stats import dirichlet
 

import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from itertools import combinations
from itertools import combinations_with_replacement
from itertools import permutations
from itertools import product


from tqdm import trange, notebook

pd.set_option('display.max_columns', None)


In [3]:
os.chdir('D:/Dropbox/Dropbox/uchicago_fourth/uncertaintyInequality')
os.getcwd()

'D:\\Dropbox\\Dropbox\\uchicago_fourth\\uncertaintyInequality'

# Creating Wide Panel

In [3]:
HH_Consumption_dir = 'pyramid/consumption/'
zipFiles = os.listdir(HH_Consumption_dir)
zipFiles

['consumption_pyramids_20140131_MS_rev_csv.zip',
 'consumption_pyramids_20140228_MS_rev_csv.zip',
 'consumption_pyramids_20140331_MS_rev_csv.zip',
 'consumption_pyramids_20140430_MS_rev_csv.zip',
 'consumption_pyramids_20140531_MS_rev_csv.zip',
 'consumption_pyramids_20140630_MS_rev_csv.zip',
 'consumption_pyramids_20140731_MS_rev_csv.zip',
 'consumption_pyramids_20140831_MS_rev_csv.zip',
 'consumption_pyramids_20140930_MS_rev_csv.zip',
 'consumption_pyramids_20141031_MS_rev_csv.zip',
 'consumption_pyramids_20141130_MS_rev_csv.zip',
 'consumption_pyramids_20141231_MS_rev_csv.zip',
 'consumption_pyramids_20150131_MS_rev_csv.zip',
 'consumption_pyramids_20150228_MS_rev_csv.zip',
 'consumption_pyramids_20150331_MS_rev_csv.zip',
 'consumption_pyramids_20150430_MS_rev_csv.zip',
 'consumption_pyramids_20150531_MS_rev_csv.zip',
 'consumption_pyramids_20150630_MS_rev_csv.zip',
 'consumption_pyramids_20150731_MS_rev_csv.zip',
 'consumption_pyramids_20150831_MS_rev_csv.zip',
 'consumption_pyrami

In [9]:
initalFile = 12 #Start at 2015
fileName = HH_Consumption_dir+ zipFiles[initalFile]

consumptionCols = ['TOTAL_EXPENDITURE',
'MONTHLY_EXPENSE_ON_FOOD',
'MONTHLY_EXPENSE_ON_INTOXICANTS',
'MONTHLY_EXPENSE_ON_CLOTHING_AND_FOOTWEAR',
'MONTHLY_EXPENSE_ON_COSMETIC_AND_TOILETRIES',
'MONTHLY_EXPENSE_ON_APPLIANCES',
'MONTHLY_EXPENSE_ON_RESTAURANTS',
'MONTHLY_EXPENSE_ON_RECREATION',
'MONTHLY_EXPENSE_ON_BILLS_AND_RENT',
'MONTHLY_EXPENSE_ON_POWER_AND_FUEL',
'MONTHLY_EXPENSE_ON_TRANSPORT',
'MONTHLY_EXPENSE_ON_COMMUNICATION_AND_INFO',
'MONTHLY_EXPENSE_ON_EDUCATION',
'MONTHLY_EXPENSE_ON_HEALTH',
'MONTHLY_EXPENSE_ON_ALL_EMIS',
'MONTHLY_EXPENSE_ON_MISCELLANEOUS',
'REASON_FOR_NON_RESPONSE',
'FAMILY_SHIFTED']

finalCols = ['HH_ID'] + consumptionCols

df = pd.read_csv(fileName, compression='zip')

if 'FAMILY_SHIFTED' not in df.columns:
        df['FAMILY_SHIFTED'] = 'N'

df = df[finalCols]
keep_same = df.columns[0]
suffix = "_"+ fileName[fileName.find('_20')+1:fileName.find('_MS')]
df.columns = ['{}{}'.format(c, '' if c in keep_same else suffix) for c in df.columns]

for f in notebook.tqdm(zipFiles[initalFile+1:71], desc='File', leave=True):
    fileName = HH_Consumption_dir+ f
    df_toMerge = pd.read_csv(fileName, compression='zip')
    if 'FAMILY_SHIFTED' not in df_toMerge.columns:
        df_toMerge['FAMILY_SHIFTED'] = 'N'

    df_toMerge = df_toMerge[finalCols]
    
    keep_same = df_toMerge.columns[0]
    suffix = "_"+ f[f.find('_20')+1:f.find('_MS')]
    df_toMerge.columns = ['{}{}'.format(c, '' if c in keep_same else suffix) for c in df_toMerge.columns]

    df = df.merge(df_toMerge,on=['HH_ID'],how='outer',validate="1:1")

    

File:   0%|          | 0/58 [00:00<?, ?it/s]

# Finishing up! and savign the HH panel

In [12]:
df.to_feather('proc_data\HH_consumption.feather')


# Creating a long panel

In [13]:
HH_Consumption_dir = 'pyramid/consumption/'
zipFiles = os.listdir(HH_Consumption_dir)
zipFiles

['consumption_pyramids_20140131_MS_rev_csv.zip',
 'consumption_pyramids_20140228_MS_rev_csv.zip',
 'consumption_pyramids_20140331_MS_rev_csv.zip',
 'consumption_pyramids_20140430_MS_rev_csv.zip',
 'consumption_pyramids_20140531_MS_rev_csv.zip',
 'consumption_pyramids_20140630_MS_rev_csv.zip',
 'consumption_pyramids_20140731_MS_rev_csv.zip',
 'consumption_pyramids_20140831_MS_rev_csv.zip',
 'consumption_pyramids_20140930_MS_rev_csv.zip',
 'consumption_pyramids_20141031_MS_rev_csv.zip',
 'consumption_pyramids_20141130_MS_rev_csv.zip',
 'consumption_pyramids_20141231_MS_rev_csv.zip',
 'consumption_pyramids_20150131_MS_rev_csv.zip',
 'consumption_pyramids_20150228_MS_rev_csv.zip',
 'consumption_pyramids_20150331_MS_rev_csv.zip',
 'consumption_pyramids_20150430_MS_rev_csv.zip',
 'consumption_pyramids_20150531_MS_rev_csv.zip',
 'consumption_pyramids_20150630_MS_rev_csv.zip',
 'consumption_pyramids_20150731_MS_rev_csv.zip',
 'consumption_pyramids_20150831_MS_rev_csv.zip',
 'consumption_pyrami

In [14]:
initalFile = 12 #Start at 2015
fileName = HH_Consumption_dir+ zipFiles[initalFile]

consumptionCols = ['TOTAL_EXPENDITURE',
'MONTHLY_EXPENSE_ON_FOOD',
'MONTHLY_EXPENSE_ON_INTOXICANTS',
'MONTHLY_EXPENSE_ON_CLOTHING_AND_FOOTWEAR',
'MONTHLY_EXPENSE_ON_COSMETIC_AND_TOILETRIES',
'MONTHLY_EXPENSE_ON_APPLIANCES',
'MONTHLY_EXPENSE_ON_RESTAURANTS',
'MONTHLY_EXPENSE_ON_RECREATION',
'MONTHLY_EXPENSE_ON_BILLS_AND_RENT',
'MONTHLY_EXPENSE_ON_POWER_AND_FUEL',
'MONTHLY_EXPENSE_ON_TRANSPORT',
'MONTHLY_EXPENSE_ON_COMMUNICATION_AND_INFO',
'MONTHLY_EXPENSE_ON_EDUCATION',
'MONTHLY_EXPENSE_ON_HEALTH',
'MONTHLY_EXPENSE_ON_ALL_EMIS',
'MONTHLY_EXPENSE_ON_MISCELLANEOUS',
'REASON_FOR_NON_RESPONSE',
'FAMILY_SHIFTED',
'MONTH_SLOT',
'MONTH']

finalCols = ['HH_ID'] + consumptionCols

df = pd.read_csv(fileName, compression='zip')

if 'FAMILY_SHIFTED' not in df.columns:
        df['FAMILY_SHIFTED'] = 'N'

df = df[finalCols]

for f in notebook.tqdm(zipFiles[initalFile+1:71], desc='File', leave=True):
    fileName = HH_Consumption_dir+ f
    df_toMerge = pd.read_csv(fileName, compression='zip')
    if 'FAMILY_SHIFTED' not in df_toMerge.columns:
        df_toMerge['FAMILY_SHIFTED'] = 'N'

    df_toMerge = df_toMerge[finalCols]
    df = pd.concat((df,df_toMerge),axis=0)

    

File:   0%|          | 0/58 [00:00<?, ?it/s]

In [15]:
df=  df.reset_index(drop=True)
df.to_feather('proc_data\HH_consumption_long.feather')


# Tests

In [75]:
df['sum_of_exp'] = df[cols[1:]].sum(axis=1)


In [77]:
a = df[(df['TOTAL_EXPENDITURE']<df['sum_of_exp']) & (df['TOTAL_EXPENDITURE']>0)]
a[['HH_ID','TOTAL_EXPENDITURE','sum_of_exp']].head(60)


# 43158770
# 90871013
# 96666847
# 93865615
# 25709277 100
# 67903443

# df[df['HH_ID']==25709277]

# 'MONTHLY_EXPENSE_ON_BREAD' ? 
# 'MONTHLY_EXPENSE_ON_CHOCOLATE_CAKE_ICE_CREAM'
# 'MONTHLY_EXPENSE_ON_ICE_CREAMS'
# 'MONTHLY_EXPENSE_ON_TEA_COFFEE'
# 6495,6460 -  77583280
# 5161, 5101
# 5273-5318 93865615

,HH_ID,TOTAL_EXPENDITURE,sum_of_exp


In [36]:
3894-3934

-40

In [9]:
for c in df.columns:
    print(c)

TOTAL_EXPENDITURE
ADJ_TOTAL_EXPENDITURE
MONTHLY_EXPENSE_ON_FOOD
ADJ_MONTHLY_EXPENSE_ON_FOOD
MONTHLY_EXPENSE_ON_INTOXICANTS
ADJ_MONTHLY_EXPENSE_ON_INTOXICANTS
MONTHLY_EXPENSE_ON_CLOTHING_AND_FOOTWEAR
MONTHLY_EXPENSE_ON_COSMETIC_AND_TOILETRIES
MONTHLY_EXPENSE_ON_SCOURERS
MONTHLY_EXPENSE_ON_OTHER_HOUSECARE_PRODUCTS
MONTHLY_EXPENSE_ON_APPLIANCES
MONTHLY_EXPENSE_ON_RESTAURANTS
ADJ_MONTHLY_EXPENSE_ON_RESTAURANTS
MONTHLY_EXPENSE_ON_RECREATION
MONTHLY_EXPENSE_ON_BILLS_AND_RENT
MONTHLY_EXPENSE_ON_POWER_AND_FUEL
ADJ_MONTHLY_EXPENSE_ON_POWER_AND_FUEL
MONTHLY_EXPENSE_ON_TRANSPORT
ADJ_MONTHLY_EXPENSE_ON_TRANSPORT
MONTHLY_EXPENSE_ON_COMMUNICATION_AND_INFO
MONTHLY_EXPENSE_ON_EDUCATION #check I think it does include booksih things 
# MONTHLY_EXPENSE_ON_BOOKS_JOURNALS
# MONTHLY_EXPENSE_ON_SCHOOL_ACADEMIC_BOOKS
# MONTHLY_EXPENSE_ON_FICTION_NON_FICTION_BOOKS
# MONTHLY_EXPENSE_ON_STATIONERY
MONTHLY_EXPENSE_ON_HEALTH
MONTHLY_EXPENSE_ON_OTHER_BEAUTY_AND_HYGIENE_PRODUCTS
MONTHLY_EXPENSE_ON_ALL_EMIS
MONTHLY_EXPENSE_ON_MISCELLANEOUS




MONTHLY_EXPENSE_ON_DOMESTIC_HELP
MONTHLY_EXPENSE_ON_MOTOR_VEHICLE_REPAIRS
MONTHLY_EXPENSE_ON_REMITTANCES_SENT
MONTHLY_EXPENSE_ON_SOCIAL_OBLIGATIONS
MONTHLY_EXPENSE_ON_RELIGIOUS_OBLIGATIONS
MONTHLY_EXPENSE_ON_PROFESSIONAL_CONSULTATION
MONTHLY_EXPENSE_ON_GENERAL_INSURANCE
MONTHLY_EXPENSE_ON_VACATION
MONTHLY_EXPENSE_ON_LIGHTING
MONTHLY_EXPENSE_ON_FURNITURE_AND_FURNISHINGS
MONTHLY_EXPENSE_ON_PAINTING_AND_RENOVATION
MONTHLY_EXPENSE_ON_UTENSILS
MONTHLY_EXPENSE_ON_POCKET_MONEY
MONTHLY_EXPENSE_ON_PETS

HH_ID
STATE
HR
DISTRICT
REGION_TYPE
STRATUM
PSU_ID
MONTH_SLOT
MONTH
RESPONSE_STATUS
REASON_FOR_NON_RESPONSE
HH_WEIGHT_MS
HH_WEIGHT_FOR_COUNTRY_MS
HH_WEIGHT_FOR_STATE_MS
HH_NON_RESPONSE_MS
HH_NON_RESPONSE_FOR_COUNTRY_MS
HH_NON_RESPONSE_FOR_STATE_MS
AGE_GROUP
OCCUPATION_GROUP
EDUCATION_GROUP
GENDER_GROUP
SIZE_GROUP
TOTAL_EXPENDITURE
ADJ_TOTAL_EXPENDITURE
MONTHLY_EXPENSE_ON_FOOD
ADJ_MONTHLY_EXPENSE_ON_FOOD
MONTHLY_EXPENSE_ON_CEREALS_AND_PULSES
MONTHLY_EXPENSE_ON_CEREALS_WHOLEGRAIN
MONTHLY_EXPENSE_ON_PROCESSED_CEREALS_AND_PULSES
MONTHLY_EXPENSE_ON_PULSES
MONTHLY_EXPENSE_ON_EDIBLE_OILS
ADJ_MONTHLY_EXPENSE_ON_EDIBLE_OILS
MONTHLY_EXPENSE_ON_GHEE
MONTHLY_EXPENSE_ON_DRY_SPICES
MONTHLY_EXPENSE_ON_VEGETABLES_AND_FRUITS
MONTHLY_EXPENSE_ON_VEGETABLES_AND_WET_SPICES
ADJ_MONTHLY_EXPENSE_ON_VEGETABLES_AND_WET_SPICES
MONTHLY_EXPENSE_ON_FRUITS
ADJ_MONTHLY_EXPENSE_ON_FRUITS
MONTHLY_EXPENSE_ON_DRY_FRUITS_AND_SAFFRON
MONTHLY_EXPENSE_ON_POTATOES_AND_ONIONS
ADJ_MONTHLY_EXPENSE_ON_POTATOES_AND_ONIONS
MONTHLY_